### Document to Structured Data

## Set Up Azure OpenAI

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()


True

### Request API

In [3]:
deployment_name = "gpt-35-turbo-16k"

# call OpenAI API with model name and prompt
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

response = client.chat.completions.create(
  model = deployment_name,
  messages = [{"role":"user","content":"who are you?"},],
  temperature=0.5,
  max_tokens=8192,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

ChatCompletion(id='chatcmpl-8mz4DfvT7Aq3woxERWeuKf3iFruAR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am an AI language model developed by OpenAI. I am here to help answer your questions and engage in conversation with you.', role='assistant', function_call=None, tool_calls=None))], created=1706685177, model='gpt-35-turbo-16k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=11, total_tokens=37))

In [5]:
print(response.choices[0].message.role)
print(response.choices[0].message.content)

assistant
I am an AI language model developed by OpenAI. I am here to help answer your questions and engage in conversation with you.


In [6]:
def request_api(messages, deployment_name):
    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version=os.getenv("OPENAI_API_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        )    
    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        temperature=0,
        max_tokens=8192,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop='###')
    return response

### Get Structured Data

In [7]:
def get_structured_data(document, prompt_postfix, deployment_id):
    content = prompt_postfix.replace('<document>', document)
    messages = [{"role":"user","content":content},]; 
    #print(messages)

    structured_data = request_api(messages, deployment_id)
    return structured_data

### Document Type: Resume

In [9]:
fname = "../data/resume.txt"

with open(fname, 'r') as f:
    document = f.readlines()

# convert list to str
document = ' '.join(document); 
#print(document)

In [10]:
# prompt
prompt_postfix = """ <document>
  \n###
  \nExtract the key sections from the resume above into json.
"""
structured_data = get_structured_data(document, prompt_postfix, deployment_name)

In [12]:
print(structured_data.choices[0].message.content)

{
  "contact": {
    "email": "chew.yean.yam@gmail.com",
    "linkedin": "www.linkedin.com/in/cyyam"
  },
  "top_skills": [
    "Research",
    "Microarray Analysis",
    "OpenCV"
  ],
  "languages": [
    "English (Native or Bilingual)",
    "Malay (Professional Working)",
    "Mandarin (Native or Bilingual)",
    "Cantonese (Native or Bilingual)"
  ],
  "certifications": [
    "Uncovering Your Authentic Self at Work",
    "Worldwide Communities - Community SME 2018",
    "Fred Kofman on Managing Conflict"
  ],
  "summary": "Over 20 years of industrial and applied research experience with global advanced technology companies and internationally renowned research institutions. Experienced in building and leading Data Science and Machine Learning specialist teams. Advises on AI strategies and identifies commercial opportunities. Experienced in incubation of new AI capabilities for products.",
  "experience": [
    {
      "company": "Microsoft",
      "title": "Principal Data and Applie